In [1]:
from helpers import *
import numpy as np
import random
import scipy
from scipy.stats import norm

In [2]:
data = np.load('ct_data.npz')
X_train = data['X_train']; X_val = data['X_val']; X_test = data['X_test']
y_train = data['y_train']; y_val = data['y_val']; y_test = data['y_test']

In [ ]:
def get_mean_and_errorbar(data):
    N = data.shape[0]
    sample_mean = 1/N * np.sum(data)
    sample_var = 1/(N-1) * np.sum((data - sample_mean)**2)
    sample_std = np.sqrt(sample_var)
    standard_error = sample_std / np.sqrt(N)
    return sample_mean, standard_error

sample_mean_y_val, standard_error_y_val = get_mean_and_errorbar(y_val)

partial_y_train = y_train[:5785]
sample_mean_y_train, standard_error_y_train = get_mean_and_errorbar(partial_y_train)

txt1 = "y_val mean with a standard error: {mean:.10f} +/- {stderr:.10f}"
print(txt1.format(mean = sample_mean_y_val, stderr = standard_error_y_val))

txt2 = "partial_y_train mean with a standard error: {mean:.10f} +/- {stderr:.10f}"
print(txt2.format(mean = sample_mean_y_train, 
                 stderr = standard_error_y_train))

We got the different sample means from y_val and partial_y_train datasets. The results demonstrate that we can't rely on the standard error bars because the sample means are random variables, and the errors are computed based on them, not the true mean. The standard error bars are misleading because one might rely on the indication to see the average locations in future CT slices. They indicate how precisely we have measured the mean of the distribution with our N samples.

In [3]:
remove_constants = []
remove_duplicates = []
unique = []
for col in range(X_train.shape[1]):  
    # remove constants
    if np.all(X_train[:,col]==X_train[:,col][0]):
        remove_constants.append(col) 
        continue
    # remove duplicates
    if list(X_train[:,col]) not in unique:
        unique.append(list(X_train[:,col]))        
    else:
        remove_duplicates.append(col)  

remove_both = remove_constants + remove_duplicates
X_train = np.delete(X_train, remove_both, axis=1)
X_val = np.delete(X_val, remove_both, axis=1)
X_test = np.delete(X_test, remove_both, axis=1)

In [340]:
X_train.shape

(40754, 373)

In [337]:
remove_constants

[59, 69, 179, 189, 351]

In [338]:
remove_duplicates

[78, 79, 188, 199, 287, 359]

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(X_train)
df.head(50)

In [ ]:
df2 = pd.DataFrame(X_all_dup_removed)
df2.head(50)

# 2. Linear regression baseline

In [5]:
def fit_linreg(X, yy, alpha):
    K = X.shape[1] + 1
    design_matrix = np.c_[X, np.ones(X.shape[0])] # add a column of ones for the bias
    alpha_identity = np.sqrt(alpha) * np.identity(K)
    design_matrix = np.vstack((design_matrix, alpha_identity)) # add regularizer
    
    design_matrix[design_matrix.shape[0]-1][K-1] = 0 # remove penalty for the bias
    augmented_y= np.hstack([yy, np.zeros(K)]) # augment vector of observations
    
    fit = np.linalg.lstsq(design_matrix, augmented_y, rcond = None)
    weights = fit[0][:-1]
    bias = fit[0][-1]
    return weights, bias

    
def rmse(true, pred):
    RSME = np.sqrt(np.square(np.subtract(true, pred)).mean())
    return RSME

alpha = 30
weights, bias = fit_linreg(X_train, y_train, alpha)
y_pred_train = np.dot(X_train, weights) + bias
y_pred_val = np.dot(X_val, weights) + bias
train_error = rmse(y_train, y_pred_train)
val_error = rmse(y_val, y_pred_val)

print('Root mean squared error on training set: {}'.format(train_error))
print('Root mean squared error on validation set: {}'.format(val_error))

Root mean squared error on training set: 0.3567565397204054
Root mean squared error on validation set: 0.4230521968394694


In [10]:
ww_gradopt, bb_gradopt = fit_linreg_gradopt(X_train, y_train, alpha)
y_pred_train = np.dot(X_train, ww_gradopt) + bb_gradopt
y_pred_val = np.dot(X_val, ww_gradopt) + bb_gradopt
train_errs = rmse(y_train, y_pred_train)
val_errs = rmse(y_val, y_pred_val)
print('Root mean squared error on training set: {}'.format(train_errs))
print('Root mean squared error on validation set: {}'.format(val_errs))

Root mean squared error on training set: 0.35675625907436664
Root mean squared error on validation set: 0.42305021197807124


In [13]:
K = X_train.shape[1]
X_train_reg = np.vstack([X_train, np.sqrt(alpha) * np.identity(K)])
y_train_reg = np.hstack([y_train, np.zeros(K)])  

ww_solver = np.linalg.solve(np.dot(X_train_reg.T, X_train_reg), 
                            np.dot(X_train_reg.T, y_train_reg))
print('Are weights by lstsq and solver the same?: {}'
      .format(np.array_equal(np.round(weights, 5), np.round(ww_solver, 5))))

Are weights by lstsq and solver the same?: False


We get the almost same root mean squared errors from our fit_linreg and the provided fit_linreg_gradopt. It is because (X.T * X) is invertible, and the number of samples is greater than or equal to the number of dimensions (N >= D), so the normal-equation is defined, and there is a unique solution for the best setting of the weights. In such a case, the gradient-based optimizer finds the same answer because the cost function is strictly convex and has one unique minimum. We can also confirm from the below sanity check that the normal-equation gives the same weights.

# 3. Invented classification tasks

In [42]:
def fit_logreg_gradopt(X, yy, alpha):
    D = X.shape[1]
    args = (X, yy, alpha)
    #init = (np.zeros(D), np.array(0))
    init = (0.01 * np.random.randn(D) / np.sqrt(D), np.array(0))
    ww, bb = minimize_list(logreg_cost, init, args)
    return ww, bb

def sigmoid(a): return 1. / (1. + np.exp(-a))

In [68]:
alpha = 30
V = []
bk = []

K = 20 # number of thresholded classification problems to fit
mx = np.max(y_train); mn = np.min(y_train); hh = (mx-mn)/(K+1)
thresholds = np.linspace(mn+hh, mx-hh, num=K, endpoint=True)
for kk in range(K):
    labels = y_train > thresholds[kk]
    # fit logistic regression to these labels
    ww, bb = fit_logreg_gradopt(X_train, labels, alpha)
    V.append(ww)
    bk.append(bb)

V = np.array(V)
bk = np.array(bk)

X_train_prob = sigmoid(np.dot(X_train, V.T) + bk)
X_val_prob = sigmoid(np.dot(X_val, V.T) + bk)
X_test_prob = sigmoid(np.dot(X_test, V.T) + bk)

print(X_train_prob.shape)

(40754, 20)


In [69]:
# fit regularized linear regression to the transformed datasets
ww, bb = fit_linreg(X_train_prob, y_train, alpha)
y_pred_train = np.dot(X_train_prob, ww) + bb
y_pred_val = np.dot(X_val_prob, ww) + bb
train_errs = rmse(y_train, y_pred_train)
val_errs = rmse(y_val, y_pred_val)
print('Root mean squared error on training set: {}'.format(train_errs))
print('Root mean squared error on validation set: {}'.format(val_errs))

Root mean squared error on training set: 0.1544105592251483
Root mean squared error on validation set: 0.25424931879375523


# 4. Small neural network

In [16]:
def init_a(D, K=1):
    np.random.seed(19)
    # Glorot and Bengio's init
    if K > 1:
        std = (2. / (K + D))**0.5
    else:
        std = (1. / K)**0.5
    ww = np.random.normal(loc=0., scale=std, size=K)
    bb = np.array(0)
    V = np.random.normal(loc=0., scale=std, size=(K, D))
    bk = std * np.random.randn(K) / np.sqrt(K)
    return (ww, bb, V, bk)

In [23]:
def init_b(D, K=1):    
    W_matrix_log
    bias_vector_log 

    ww = np.zeros(K)
    bb = np.array(0)
    V = np.zeros((K, D))
    bk = np.zeros(K)
    return (ww, bb, V, bk)

In [20]:
def train_nn(X_train, X_val, init, K, alpha=30):
    args = (X_train, y_train, alpha) 
    ww, bb, V, bk = minimize_list(nn_cost, init(X_train.shape[1], K), args)
    params = (ww, bb, V, bk)
    y_pred_train = nn_cost(params, X_train)
    y_pred_val = nn_cost(params, X_val)
    train_errs = rmse(y_train, y_pred_train)
    val_errs = rmse(y_val, y_pred_val)
    return train_errs, val_errs

In [72]:
# fit NN with a sensible random initialization
train_errs, val_errs = train_nn(X_train, X_val, init_a, K=20)
print('Init_a: RMSE on training set: {}'.format(train_errs))
print('Init_a: RMSE on validation set: {}'.format(val_errs))

Init_a: RMSE on training set: 0.13983192980941112
Init_a: RMSE on validation set: 0.2711542448010652


In [71]:
# fit NN with the parameters initialized using the fits made in Q3
args = (X_train, y_train, alpha) 
ww, bb, V, bk = minimize_list(nn_cost, (ww, bb, V, bk), args)
params = (ww, bb, V, bk)
y_pred_train = nn_cost(params, X_train)
y_pred_val = nn_cost(params, X_val)
train_errs = rmse(y_train, y_pred_train)
val_errs = rmse(y_val, y_pred_val)
print('Init_a: RMSE on training set: {}'.format(train_errs))
print('Init_a: RMSE on validation set: {}'.format(val_errs))

RMSE on training set using prefitted weights: 0.1387897525445384
RMSE on validation set using prefitted weights: 0.27079711742451973


The strategy (a) that initializes the weights randomly works better than the one (b) used in Q3, which sets all the weights to zero. In (b), unlike (a), our neural network function only depends on a single linear combination of the inputs (X_train_prob) because our K hidden units start with the same parameters so that each unit gets the same gradient vector.

In [27]:
args = (X_train, y_train, alpha) 
minimize_list(nn_cost, init_b(X_train.shape[1], K=20), args)[2]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
minimize_list(nn_cost, init_a(X_train.shape[1], K=20), args)[2]

array([[-1.05383299e-01,  6.01259784e-03, -3.42595177e-02, ...,
         1.52258932e-01, -2.11320448e-02,  1.34620981e-02],
       [ 6.41236638e-03,  1.01879122e-02,  2.05937008e-03, ...,
         1.12573064e-03,  1.83921041e-03, -4.90584301e-04],
       [ 3.40601229e-02,  3.19521382e-02,  5.00998874e-03, ...,
        -6.46171803e-03,  4.30461944e-03,  8.46586084e-04],
       ...,
       [ 6.72947646e-03,  1.14041061e-02,  2.30239212e-03, ...,
         1.65891885e-03,  2.22419736e-03, -6.06355079e-04],
       [ 3.37371538e-03,  4.88399052e-03,  9.24689633e-04, ...,
         3.82506420e-06,  5.51003745e-04,  1.18439567e-05],
       [ 6.98182734e-03,  1.14713770e-02,  2.31584395e-03, ...,
         1.61313213e-03,  2.16697232e-03, -5.95725651e-04]])

Jointly fitting the neural network works better because the Q3 model only fits weights for a single linear combination as a feedforward neural network. On the other hand, the neural network model fits parameters for one hidden layer with K=20 hidden units using a gradient-based optimizer, reducing the cost by updating the parameters. Although the cost function is not convex, this optimization procedure using forward and back propagations likely to find the local optima with a better cost and generalized better on the validation data.

# Bayesian optimisation

In [286]:
# def train_nn_reg(alpha):
#     train_errs, val_errs = train_nn(X_train_prob, X_val_prob, init_a, alpha=alpha)
#     return val_errs

In [329]:
# val_errs = 0.26705441493687515 # Q4 validation error

# X_rest = np.arange(0, 50, 0.02)
# X_obs = np.random.choice(alphas, size=3)
# idx = [np.where(alphas==x)[0][0] for x in X_obs]

# yy = np.array([np.log(val_errs) - np.log(train_nn_reg(alpha)) for alpha in X_obs])
# rest_cond_mu, rest_cond_cov = helpers.gp_post_par(X_rest, X_obs, yy)
# PI_alpha = np.array([norm.cdf((rest_cond_mu[i] - np.max(yy))/np.sqrt(rest_cond_cov[i][i])) for i in idx])

In [330]:
# # 5 iterations of probability of improvement acquisition function
# alphas_valerrs = []
# for i in range(5):     
#     rmses = [[train_nn_reg(alpha), alpha] for alpha in PI_alpha]
#     alphas_valerrs.append(rmses)
#     yy = np.array([np.log(val_errs) - np.log(rmse[0]) for rmse in rmses])
#     rest_cond_mu, rest_cond_cov = helpers.gp_post_par(X_rest, PI_alpha, yy)
#     idx = [np.where(alphas == np.round(alpha, 1)) for alpha in PI_alpha]
#     PI_alpha = np.array([norm.cdf((rest_cond_mu[i[0][0]] - np.max(yy))/
#                                   np.sqrt(rest_cond_cov[i[0][0]][i[0][0]])) for i in idx])  

In [331]:
# print("Best PI and its alpha")
# i = 1
# best_PI_alpha = None
# best_PI_alphas = []
# for alpha_valerr in np.array(alphas_valerrs):
#     best_PI = np.min(alpha_valerr)
#     best_PI_alpha = alpha_valerr[np.where(alpha_valerr[:,0] == best_PI)]
#     best_PI_alphas.append(best_PI_alpha)
#     print(i, best_PI_alpha[0][0], best_PI_alpha[0][1])
#     i+=1

Best PI and its alpha
1 0.26299832114162164 0.4764329186620903
2 0.26300938166190846 0.4819881472031914
3 0.26289831177491924 0.4744821046215118
4 0.26301297585091915 0.4783196769420628
5 0.2629760501084887 0.48150241928503074


In [332]:
# best_PI_alphas = np.array(best_PI_alphas)
# best_PI = np.min(best_PI_alphas)
# best_alpha = best_PI_alphas[np.where(best_PI_alphas[:,0] == best_PI)][0][1]
# print("Best alpha:", best_alpha)

Best alpha: 0.4744821046215118


In [334]:
# args = (X_train_prob, y_train, best_alpha) 
# ww, bb, V, bk = helpers.minimize_list(helpers.nn_cost, init_a(X_train_prob), args)
# params = (ww, bb, V, bk)
# y_pred_val = helpers.nn_cost(params, X_val_prob)
# y_pred_test = helpers.nn_cost(params, X_test_prob)
# val_errs = rmse(y_val, y_pred_val)
# test_errs = rmse(y_test, y_pred_test)
# print('RMSE on validation set: {}'.format(val_errs))
# print('RMSE on test set: {}'.format(test_errs))

RMSE on validation set: 0.2630035421721344
RMSE on test set: 0.27425911987829765


# Bayesian optimization2

In [29]:
from scipy.stats import norm

def train_nn_reg(alpha):
    train_errs, val_errs = train_nn(X_train, X_val, init_a, K=20, alpha=alpha)
    return val_errs

# def train_nn_reg(X_train, y_train, test_input, test_output, alpha): 
    
#     K = 20
#     D = X_train.shape[1]
    
#     # initialize weights and args
#     ww_init = glorot_init_normal((K,))
#     W_init = glorot_init_normal((K,D))
#     bb_init = 0
#     B_init = np.zeros((K,))

#     args = (X_train, y_train, alpha)
#     init = (ww_init, bb_init, W_init, B_init)

#     # get fitted weights and biases
#     ww, bb, W, B = minimize_list(nn_cost, init, args)
#     params = (ww, bb, W, B)
    
#     y_pred = nn_cost(params, test_input, None, alpha)
#     error = rmse(test_output, y_pred)
    
#     return error
    

def prob_improvement(mean_alpha, std_alpha, yy):
    max_y = max(yy)
    pi_alpha = norm.cdf((mean_alpha - max_y) / std_alpha)
    return pi_alpha


def maximise_acquisition_function(alpha_rest, alpha_obs, y_obs):
    # get GP posterior evaluated at test points
    mean_rest, cov_rest = gp_post_par(alpha_rest, alpha_obs, y_obs)
    std_rest = np.sqrt(np.diag(cov_rest))

    # initialize values
    alpha_best = 0
    max_pi = 0
    
    # find alpha that yields the best PI
    for alpha, mean, std in zip(alpha_rest, mean_rest, std_rest):
        pi_alpha = prob_improvement(mean, std, y_obs)
        
        if pi_alpha > max_pi:
            alpha_best = alpha
            max_pi = pi_alpha
            
    return alpha_best, max_pi

In [30]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

# validation RMSE from Q4 using random initialization
rmse_baseline = 0.2708803021948929

# get three observations from training the neural network at three alpha values
alpha_obs = np.array([10, 25, 40])

rmse_val = []
for a in alpha_obs:
    val_err = train_nn_reg(a)
    rmse_val.append(val_err)
    
# subtract your alpha-observed log RMSEs from the log of this baseline and take the resulting values as y^(1), ...,  y^(3)
y_obs = np.log(rmse_baseline) - np.log(rmse_val)

alpha_rest = np.arange(0, 50, 0.02)

# remove already observed values of alpha
alpha_rest = np.setdiff1d(alpha_rest, alpha_obs)

results = []
for i in range(5):
    # find alpha with highest PI
    alpha_best, max_pi = maximise_acquisition_function(alpha_rest, alpha_obs, y_obs)
    
    # add to observed locations and remove from test locations
    alpha_obs = np.append(alpha_obs, alpha_best)
    alpha_rest = np.setdiff1d(alpha_rest, alpha_obs)
    
    # train using this alpha
    val_rmse_alpha = train_nn_reg(alpha_best)
    y_alpha = np.log(rmse_baseline) - np.log(val_rmse_alpha)
    
    # add to observed values
    y_obs = np.append(y_obs, y_alpha)
    
    results.append((alpha_best, max_pi, val_rmse_alpha))    

In [31]:
for i in range(len(results)):
    result = results[i]
    print("ITERATION: {}, ALPHA: {}, PI: {}, VAL_RMSE: {}".format(i+1, result[0], result[1], result[2]))

ITERATION: 1, ALPHA: 10.620000000000001, PI: 0.44039086203373706, VAL_RMSE: 0.2627446293192474
ITERATION: 2, ALPHA: 8.72, PI: 0.44338366078962627, VAL_RMSE: 0.25830412503646893
ITERATION: 3, ALPHA: 6.38, PI: 0.40677517263838653, VAL_RMSE: 0.25577169609575706
ITERATION: 4, ALPHA: 5.3, PI: 0.4062555770452248, VAL_RMSE: 0.2575502217047426
ITERATION: 5, ALPHA: 5.32, PI: 0.39441565302052106, VAL_RMSE: 0.2560261364785499


In [36]:
# find best alpha and report its errors
best_alpha = results[2][0]
args = (X_train, y_train, best_alpha) 
ww, bb, V, bk = minimize_list(nn_cost, init_a(X_train.shape[1], K=20), args)
params = (ww, bb, V, bk)
y_pred_test = nn_cost(params, X_test)
test_errs = rmse(y_test, y_pred_test)
print('Best alpha: {}'.format(best_alpha))
print('Best RMSE on validation set: {}'.format(results[2][2]))
print('Best RMSE on test set: {}'.format(test_errs))

Best alpha: 6.38
Best RMSE on validation set: 0.25577169609575706
Best RMSE on test set: 0.28963171053159736


We have improved the validation RMSE from the Q4 model by 0.0151. The observation noise is coming from the random initialization of weights in the neural network.

# What's next

In [50]:
# QUESTION 6

def one_hot_encode(labels, K):
    # labels: target outputs
    # K: number of classes
    N = labels.shape[0]
    
    mx = np.max(labels); mn = np.min(labels); hh = (mx-mn)/K
    #thresholds = np.arange(mn+hh, mx, hh)
    thresholds = np.linspace(mn+hh, mx-hh, num=K-1, endpoint=True)
    # find which class each y-value belongs to
    y_class = np.searchsorted(thresholds, labels)
    encoding = np.zeros((N, K))
    # set corresponding class to 1
    encoding[np.arange(N), y_class] = 1
    
    return encoding, y_class

In [51]:
def softmax(z):
    # subtract the max from the exponent for numerical stability
    exp_z = np.exp(z - z.max(-1)[:, None])
    
    # normalise as probabilities
    prob = exp_z /exp_z.sum(-1)[:, None]
    
    return prob

In [52]:
def init_glorot(D, K):
    np.random.seed(19)
    # Glorot and Bengio's init
    if K > 1:
        std = (2. / (K + D))**0.5
    else:
        std = (1. / K)**0.5

    ww = np.random.normal(loc=0., scale=std, size=(K, D))
    bb = np.zeros((K,))
    return (ww, bb)

In [53]:
def softmax_reg_cost(params, X, yy=None, alpha=None):

    # Unpack parameters from list
    W, b = params
    
    # FORWARD COMPUTATION
    Z = np.dot(X, W.T) + b[None,:] # N,K
    F = softmax(Z)
    
    # at test time
    if yy is None:
        # user wants prediction rather than training signal:
        return F
    
    # compute cross-entropy error
    # -np.mean(np.sum(targets * np.log(probs), axis=1))
    E = -np.mean(np.sum(yy * np.log(F), axis=1)) + alpha*np.sum(W*W)
    #print('Error:',E) 

    
    # BACKPROPOGATION
    
    # derivate of E with respect to h2
    # (probs - targets) / outputs.shape[0]
    
#     probs = np.exp(F - F.max(-1)[:, None])
#     probs /= probs.sum(-1)[:, None]
#     F_bar = (probs - yy) / F.shape[0]
    F_bar = (F - yy) / yy.shape[0]
    
    # derivate of E with respect to z2
    
    # softmax
    # outputs * (grads_wrt_outputs - (grads_wrt_outputs * outputs).sum(-1)[:, None]))
    Z_bar = F * (F_bar - (F_bar * F).sum(-1)[:, None])
    
    # np.dot(grads_wrt_outputs.T, inputs)               
    W_bar = np.dot(Z_bar.T, X) + 2*alpha*W # 2*alpha*W is the gradient of the penalty term
    
    # np.sum(grads_wrt_outputs, axis=0)
    b_bar = np.sum(Z_bar, axis=0) # scalar

    return E, (W_bar, b_bar)

In [89]:
def make_softmax_units(K):
    alpha=0.0001
    D = X_train.shape[1]   
    # pre-process labels
    y_encoded, _ = one_hot_encode(y_train, K)

    args = (X_train, y_encoded, alpha)
    W, b = minimize_list(softmax_reg_cost, init_glorot(D, K), args)
    params = (W, b)
    X_train_trans = softmax_reg_cost(params, X_train, None, alpha)
    X_val_trans = softmax_reg_cost(params, X_val, None, alpha)
    X_test_trans = softmax_reg_cost(params, X_test, None, alpha)
    
    return X_train_trans, X_val_trans, X_test_trans

In [46]:
def make_sigmoid_units(K, alpha):
    X_train_ = []; X_val_ = []; X_test_ = []   

    mx = np.max(y_train); mn = np.min(y_train); hh = (mx-mn)/(K+1)
    thresholds = np.linspace(mn+hh, mx-hh, num=K, endpoint=True)
    for kk in range(K):
        labels = y_train > thresholds[kk]
        # fit logistic regression to these labels
        ww, bb = fit_logreg_gradopt(X_train, labels, alpha)
        X_train_.append(sigmoid(np.dot(X_train, ww.T) + bb))
        X_val_.append(sigmoid(np.dot(X_val, ww.T) + bb))
        X_test_.append(sigmoid(np.dot(X_test, ww.T) + bb))
    
    return np.array(X_train_).T, np.array(X_val_).T, np.array(X_test_).T

In [108]:
def train_linreg(X_train, X_val, alpha=30):
    ww, bb = fit_linreg(X_train, y_train, alpha)
    y_pred_train = np.dot(X_train, ww) + bb
    y_pred_val = np.dot(X_val, ww) + bb
    train_error = rmse(y_train, y_pred_train)
    val_error = rmse(y_val, y_pred_val)
    return train_error, val_error

In [92]:
X_train_trans, X_val_trans,_ = make_softmax_units(32)
train_errs, val_errs = train_small_nn(X_train_trans, X_val_trans, alpha=30)
val_errs

0.21157779795386927

In [ ]:
# normal
Number of classes: 20
Training error: 0.2409605615667869
Validation error: 0.2503275903968519

# unif
Number of classes: 20
Training error: 0.2482897062704253
Validation error: 0.25643079776478994

In [94]:
ks = [10, 20, 30, 40, 50]
alpha = 30
errs_sig = []
errs_sof = []

for K in ks:
    X_train_sig, X_val_sig,_ = make_sigmoid_units(K, alpha)
    train_errs_sig, val_errs_sig = train_small_nn(X_train_sig, X_val_sig, alpha)
    errs_sig.append(np.round(val_errs_sig, 3))
    
    X_train_sof, X_val_sof,_ = make_softmax_units(K)
    train_errs_sof, val_errs_sof = train_small_nn(X_train_sof, X_val_sof, alpha)
    errs_sof.append(np.round(val_errs_sof, 3))
    
print('Sigmoid units RMSEs on validation set:', errs_sig)
print('Softmax units RMSEs on validation set:', errs_sof)

Sigmoid units RMSEs on validation set: [0.256, 0.254, 0.255, 0.255, 0.255]
Softmax units RMSEs on validation set: [0.249, 0.272, 0.24, 0.245, 0.213]


In [99]:
ks = [10, 20, 30, 40, 50, 60, 70, 80]
alpha = 30
errs_sof_linreg = []
errs_sof_nn = []
for K in ks:    
    X_train_sof_linreg, X_val_sof_linreg,_ = make_softmax_units(K)
    train_errs_sof_linreg, val_errs_sof_linreg = train_small_nn(X_train_sof_linreg, X_val_sof_linreg, alpha)
    errs_sof_linreg.append(np.round(val_errs_sof_linreg, 3))
    
    X_train_sof_nn, X_val_sof_nn,_ = make_softmax_units(K)
    train_errs_sof_nn, val_errs_sof_nn = train_nn(X_train_sof_nn, X_val_sof_nn, init_a, K, alpha)
    errs_sof_nn.append(np.round(val_errs_sof_nn, 3))    
    
print('Softmax + fit_linreg RMSEs on validation set:', errs_sof_linreg)
print('Softmax + nn_cost RMSEs on validation set:', errs_sof_nn)

Sigmoid units RMSEs on validation set: [0.249, 0.272, 0.24, 0.245, 0.213, 0.225, 0.233, 0.234]
Softmax units RMSEs on validation set: [0.249, 0.272, 0.238, 0.239, 0.209, 0.218, 0.222, 0.222]


In [105]:
# evaluate the best K on test set
best_K = 49
alpha = 30
X_train_sof_nn, X_val_sof_nn, X_test_sof_nn = make_softmax_units(best_K)
args = (X_train_sof_nn, y_train, alpha) 
ww, bb, V, bk = minimize_list(nn_cost, init_a(X_train_sof_nn.shape[1], best_K), args)
params = (ww, bb, V, bk)
y_pred_val = nn_cost(params, X_val_sof_nn)
y_pred_test = nn_cost(params, X_test_sof_nn)
val_error = rmse(y_val, y_pred_val)
test_error = rmse(y_test, y_pred_test)
print('Best RMSE on validation set: {}'.format(val_error))
print('Best RMSE on test set: {}'.format(test_error))

Best RMSE on validation set: 0.22013729663098597
Best RMSE on test set: 0.22111682986948125


In [106]:
np.arange(20, 100, 20)

array([20, 40, 60, 80])

In [109]:
# compare softmax with sigmoid models
ks = np.arange(20, 100, 20)
alpha = 30
errs_sig = []
errs_sof = []

for K in ks:
    X_train_sig, X_val_sig,_ = make_sigmoid_units(K, alpha)
    train_errs_sig, val_errs_sig = train_linreg(X_train_sig, X_val_sig, alpha)
    errs_sig.append((K, val_errs_sig))
    
    X_train_sof, X_val_sof,_ = make_softmax_units(K)
    train_errs_sof, val_errs_sof = train_linreg(X_train_sof, X_val_sof, alpha)
    errs_sof.append((K, val_errs_sof))
    
    
errs_sig_print = [np.round(result[1], 3) for result in errs_sig]
errs_sof_print = [np.round(result[1], 3) for result in errs_sof]
    
print('Sigmoid units RMSEs on validation set:', errs_sig_print)
print('Softmax units RMSEs on validation set:', errs_sof_print)

Sigmoid units RMSEs on validation set: [0.254, 0.255, 0.254, 0.255]
Softmax units RMSEs on validation set: [0.272, 0.245, 0.225, 0.234]


In [ ]:
# focus on around 50 to search for the best K
# alpha = 30
# val_errs = []
# for K in range(45, 65, 1):   
#     X_train_sof, X_val_sof, _ = make_softmax_units(K)
#     train_errs_sof, val_errs_sof = train_small_nn(X_train_sof, X_val_sof, alpha)
#     val_errs.append([K, np.round(val_errs_sof, 5)])

# Update K with GP

In [234]:
def train_nn_softmax_reg(X_train, y_train, test_input, test_output, K, alpha):     
    X_train_trans, X_val_trans = make_softmax_units(K)
    train_errs, val_errs = train_nn(X_train_trans, X_val_trans, init_a, alpha)    
    return val_errs

In [235]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

alpha = 0.0001
# validation RMSE from Q6 using softmax units net
rmse_baseline = 0.2102293480091851

# get three observations from training the neural network at three alpha values
k_obs = np.array([25, 30, 35])

rmse_val = []
for K in k_obs:
    #val_err = train_nn_reg(X_train, y_train, X_val, y_val, a)
    val_err = train_nn_softmax_reg(X_train, y_train, X_val, y_val, K, alpha)
    rmse_val.append(val_err)
    
# subtract your alpha-observed log RMSEs from the log of this baseline and take the resulting values as y^(1), ...,  y^(3)
y_obs = np.log(rmse_baseline) - np.log(rmse_val)

k_rest = np.arange(2, 64, 1)

# remove already observed values of alpha
k_rest = np.setdiff1d(k_rest, k_obs)

results = []
for i in range(5):
    # find alpha with highest PI
    k_best, max_pi = maximise_acquisition_function(k_rest, k_obs, y_obs)
    #print(alpha_best)
    
    # add to observed locations and remove from test locations
    k_obs = np.append(k_obs, k_best)
    k_rest = np.setdiff1d(k_rest, k_obs)
    
    # train using this alpha
#     val_rmse_alpha = train_nn_reg(X_train, y_train, X_val, y_val, alpha_best)
    val_rmse_k = train_nn_softmax_reg(X_train, y_train, X_val, y_val, k_best, alpha)
    y_k = np.log(rmse_baseline) - np.log(val_rmse_k)
    
    # add to observed values
    y_obs = np.append(y_obs, y_k)
    
    results.append((k_best, max_pi, val_rmse_k))
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [237]:
for i in range(len(results)):
    result = results[i]
    print("ITERATION: {}, K: {}, PI: {}, VAL_RMSE: {}".format(i+1, result[0], result[1], result[2]))

ITERATION: 1, K: 63, PI: 0.8747222916981918, VAL_RMSE: 0.24038823233935383
ITERATION: 2, K: 2, PI: 0.8747197928491879, VAL_RMSE: 0.5530904576893079
ITERATION: 3, K: 49, PI: 0.8670397889152548, VAL_RMSE: 0.22941343819267312
ITERATION: 4, K: 17, PI: 0.7431601431238777, VAL_RMSE: 0.2773592566033777
ITERATION: 5, K: 44, PI: 0.648029614599473, VAL_RMSE: 0.22955263112545585
